In [660]:
import math
import re
import string
from dataclasses import dataclass, field
from itertools import product

import matplotlib.pyplot as plt
import numpy as np

import utils

## Day 3: Gear Ratios

[#](https://adventofcode.com/2023/day/3) - We have an engine schematic where any number adjacent to a symbol is a part number. Add up all the part numbers to find the missing part.

A symbol is any character which is not a number or a period.

In [328]:
test: str = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

inp = utils.get_input(3)

This is a simple 2d grid, the twist here is that we're not just looking at adjacent cells, as the number can have multiple digits, e.g 633 would count as a part num, not just 63 (the bits adjacent to a cell).

I'm going to deal with this by making a grid class which does basic grid operations - gets a number, finds adjacent cells etc. I've coded this before in past AOC's, but as an exercise doing it again here.

To get a list of all symbols, assuming everything not a digit or a `.` is a symbol:

In [347]:
[char for char in set(inp) if char != "." and not char.isdigit()]

['+', '\n', '$', '/', '&', '%', '*', '-', '@', '=', '#']

This is a bit tricky... I was thinking of iterating through each char, checking if its a digit next to a symbol and and adding it to a list, but the twist in this problem is that continious numbers count as one number, e.g the first number is 467.

In [662]:
@dataclass
class Grid:
    inp: str
    symbols: list = field(init=False)
    grid: list = field(init=False)
    symbol_positions: list = field(init=False)
    parts: list = field(init=False)
    rows: int = field(init=False)
    cols: int = field(init=False)
    num_positions: set = field(default_factory=set)

    def __post_init__(self):
        self.grid = self.make_grid(self.inp)
        self.rows = len(self.grid)
        self.cols = len(self.grid[0])
        self.symbols = [
            char for char in set(self.inp) if char != "." and not char.isdigit()
        ]
        self.symbol_positions = self.find_symbols(self.grid)
        self.part_numbers = self.get_parts(self.grid)

    def make_grid(self, inp: str = test):
        """transforms inp str into a list of lists"""
        grid = []
        for row in inp.strip().splitlines():
            grid.append([char for char in row])
        return grid

    def find_symbols(self, grid):
        """returns a list of positions where symbols are present"""
        symbol_positions = []
        for y, row in enumerate(grid):
            for x, char in enumerate(row):
                if char in self.symbols:
                    symbol_positions.append((x, y))
        return symbol_positions

    def get_adjacent(self, pos, diagnols=True):
        """returns a list of adjacent positions for one position"""
        x, y = pos
        xx = [n for n in range(x - 1, x + 2) if n >= 0 and n < self.rows]
        yy = [n for n in range(y - 1, y + 2) if n >= 0 and n < self.cols]

        return list([p for p in product(xx, yy) if p != pos])

    def get_parts(self, grid):
        """returns a list of all valid part numbers"""
        part_nums = []

        for y, row in enumerate(grid):
            for match in re.finditer(r"\d+", "".join(row)):
                if match:
                    adjacents = set()
                    for x in range(match.start(), match.end()):
                        pos = (x, y)
                        adjacents.update(self.get_adjacent(pos))
                    syms = [
                        self.get_char(pos)
                        for pos in adjacents
                        if self.get_char(pos) in self.symbols
                    ]
                    if syms:
                        part_nums.append(int(match.group()))

        return part_nums

    def get_gears(self, grid, N=2):
        """returns list of gears, defined as two numbers adjacent to one symbol"""
        gear_ratios = []
        pass

    def get_char(self, pos):
        """returns char at a pos defined as (x, y)"""
        x, y = pos
        assert x < self.rows and y < self.cols, f"Out of range {x=} {y=}"
        try:
            return self.grid[y][x]
        except:
            raise ValueError(f"{pos=} out of range")

    def get_full_number(self, pos):
        """returns the full number at a pos along with list of coordinates"""
        if not (num := self.get_char(pos)).isdigit():
            return False, False
        x, y = pos
        seen = set()
        seen.add(pos)

        # check to left of pos
        for xi in range(x - 1, -1, -1):
            pos = (xi, y)
            if not (char := self.get_char(pos)).isdigit():
                break
            else:
                num = char + num
                seen.add(pos)

        # check to right of pos
        for xi in range(x + 1, g.cols):
            pos = (xi, y)
            if not (char := self.get_char(pos)).isdigit():
                break
            else:
                num += char
                seen.add(pos)

        return int(num), seen


g = Grid(test)
assert sum(Grid(test).part_numbers) == 4361
sum(Grid(inp).part_numbers)

527446

## Part 2

A gear is a `*` symbol which is adjacent to exactly two part numbers.

A gear ratio is those two part numbers multiplied together. Find all the gears, calculate the gear ratio and sum them up.

The test input has 3 stars, 2 of which are next to exactly two numbers, forming a gear. So the algo is:

* find all stars
* check which ones have exactly two numbers adjacent, multiply those numbers and save to a list
* sum up the list of gears

This solution is a bit ugly, some improvement ideas below:

In [657]:
def day_2(inp=test):
    g = Grid(inp)
    gears = []

    for y, row in enumerate(g.grid):
        for match in re.finditer(r"\*", "".join(row)):
            numbers = []
            number_positions = set()
            pos = (match.start(), y)
            adj = g.get_adjacent(pos)
            # print(pos, g.get_char(pos), adj)
            for p in adj:
                if g.get_char(p).isdigit():
                    n, positions = g.get_full_number(p)
                    if number_positions.isdisjoint(positions):
                        numbers.append(n)
                        number_positions.update(positions)

            if len(numbers) == 2:
                gears.append(math.prod(numbers))

    return sum(gears)


assert day_2(test) == 467835  # example answer
day_2(inp)

73201705

Some improvements:

* use re to get all numbers in a string along with their start/stop positions, instead of looking to the left and right of a number to grab the full number and its positions.